How to use autograd

In [3]:
import autograd.numpy as np
from autograd import grad

def taylor_sine(x):
    ans = currterm = x
    i = 0

    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2 / ((2*i+3)*(2*i+2))
        ans = ans + currterm
        i += 1

    return ans

grad_sine = grad(taylor_sine)
print(f"Gradient of sin(pi) is {grad_sine(np.pi)}")


Gradient of sin(pi) is -0.9998995297042174


Complete example of autograd application on logistic regression

In [4]:
import autograd.numpy as np
from autograd import grad

def sigmoid(x):
    return 0.5*(np.tanh(x/2.)+1)

# y = sigmoid(xw)
def logistic_predictions(weights, inputs):
    return sigmoid(np.dot(inputs, weights))

# Cost function
def training_loss(weights):
    preds = logistic_predictions(weights, inputs)

    # Target is 0 or 1
    label_probabilities = preds * targets + (1-preds)*(1-targets)
    return -np.sum(np.log(label_probabilities))



inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])

targets = np.array([True, True, False, True])


training_gradient_fun = grad(training_loss)

weights = np.array([0.0,0.0,0.0])

print(f"Initial loss: {training_loss(weights)}")

# Epoch = 100 and alpha=0.01
for i in range(100):
    weights -= training_gradient_fun(weights) * 0.01


print(f"Trained loss: {training_loss(weights)}")

Initial loss: 2.772588722239781
Trained loss: 0.026923541797959805


Extend autograd by defining your own primitive and its gradient

In [6]:
# Code found here: https://github.com/HIPS/autograd/blob/master/examples/define_gradient.py

import autograd.numpy as np
from autograd.extend import primitive, defvjp

@primitive
def logsumexp(x):

    # Numerically stable log(sum(exp(x)))

    max_x = np.max(x)

    return max_x + np.log(np.sum(np.exp(x-max_x)))

def logsumexp_vjp(ans, x):
    x_shape = x.shape

    return lambda g: np.full(x_shape, g)* np.exp(x - np.full(x.shape, ans))

defvjp(logsumexp, logsumexp_vjp)


# Now we can use logsumexp anywhere, including inside of a larger function that we wanna differentiate
def example_func(y):
    z = y**2
    lse = logsumexp(z)

    return np.sum(lse)

grad_of_example = grad(example_func)
print(f"Gradient: {grad_of_example(np.array([1.5, 6.7, 1e-10]))}")

print(""" This module defines VJPs for
each NumPy op. Each line is a call to defvjp which is a thin wrapper
which just adds stuff to a Python dict which stores all the VJP functions.
For each op, we need to specify a VJP for each of its arguments. Exercise: write VJPs for division
and elementwise log.
The VJP is
represented as a function which takes in the output error signal g, the value
of the node ans, and the arguments to the op (which, remember, are Node
instances). The function returns the input error signal for the corresponding
argument.""")

Gradient: [9.09503860e-19 1.34000000e+01 6.39073344e-30]
 This module defines VJPs for
each NumPy op. Each line is a call to defvjp which is a thin wrapper
which just adds stuff to a Python dict which stores all the VJP functions.
For each op, we need to specify a VJP for each of its arguments. Exercise: write VJPs for division
and elementwise log.
The VJP is
represented as a function which takes in the output error signal g, the value
of the node ans, and the arguments to the op (which, remember, are Node
instances). The function returns the input error signal for the corresponding
argument.
